In [ ]:
!conda info --envs

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd 
import SEACells

In [ ]:
files = !ls separated_data/

In [ ]:
files

In [ ]:
for x in files:
    ad = sc.read_h5ad(f"separated_data/{x}")
    # Copy the counts to ".raw" attribute of the anndata since it is necessary for downstream analysis
    # This step should be performed after filtering 
    raw_ad = sc.AnnData(ad.X)
    raw_ad.obs_names, raw_ad.var_names = ad.obs_names, ad.var_names
    ad.raw = raw_ad
    
    # Normalize cells, log transform and compute highly variable genes
    sc.pp.normalize_per_cell(ad)
    sc.pp.log1p(ad)
    sc.pp.highly_variable_genes(ad, n_top_genes=1500)
    
    # Compute principal components - 
    # Here we use 50 components. This number may also be selected by examining variance explaint
    sc.tl.pca(ad, n_comps=50, use_highly_variable=True)
    
    sc.pp.neighbors(ad, n_neighbors=10, n_pcs=40)
    sc.tl.umap(ad)
    
    ## User defined parameters
    ## Core parameters 
    n_SEACells = int(round(len(ad.obs)/75))
    build_kernel_on = 'X_pca' # key in ad.obsm to use for computing metacells
                              # This would be replaced by 'X_svd' for ATAC data

    ## Additional parameters
    n_waypoint_eigs = 10 # Number of eigenvalues to consider when initializing metacells
    
    model = SEACells.core.SEACells(ad, 
                  build_kernel_on=build_kernel_on, 
                  n_SEACells=n_SEACells, 
                  n_waypoint_eigs=n_waypoint_eigs,
                  convergence_epsilon = 1e-5)
    
    model.construct_kernel_matrix()
    M = model.kernel_matrix
    
    # Initialize archetypes
    model.initialize_archetypes()
    
    model.fit(min_iter=10, max_iter=100)
    
    ad.obs["GSM_ID_string"] = ad.obs["GSM_ID"].astype(str)
    ad.obs["SEACell_string"] = ad.obs["SEACell"].astype(str)
    ad.obs["SEACell_ID"] = ad.obs["SEACell_string"]+"_"+ad.obs["GSM_ID_string"]
    
    ad.write_h5ad(f"data_with_seacell_id/{x[:-9]}_seacell_id.h5ad")

In [ ]:
!ls data_with_seacell_id/

In [ ]:
out = []

In [ ]:
data = !ls data_with_seacell_id/

In [ ]:
data

In [ ]:
for x in data:
    adata = sc.read_h5ad(f"data_with_seacell_id/{x}")
    out.append(adata)

In [ ]:
len(out)

In [ ]:
out

In [ ]:
adata = sc.concat(out, join="outer")

In [ ]:
adata

In [ ]:
adata.write_h5ad("hgsoc_with_SEACell_ID.h5ad")